In [1]:
from django.template import Context

from django.template.base import Token
from django.template.base import Parser
from django.template.base import Template
from django.template.base import TokenType

from django.core.management import call_command

from wagtail_srcset.templatetags.wagtail_srcset_tags import srcset_image

In [2]:
from django.core.files.uploadedfile import SimpleUploadedFile

from wagtail.images.models import Image as WagtailImage

# setup db

In [3]:
call_command("migrate")

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions, sites, taggit, wagtailadmin, wagtailcore, wagtaildocs, wagtailembeds, wagtailforms, wagtailimages, wagtailredirects, wagtailsearch, wagtailusers
Running migrations:
  No migrations to apply.


# create image

In [4]:
import io

from PIL import Image



def create_small_rgb():
    # this is a small test jpeg
    img = Image.new('RGB', (200, 200), (255, 0, 0, 0))
    return img


def small_jpeg_io():
    rgb = create_small_rgb()
    im_io = io.BytesIO()
    rgb.save(im_io, format="JPEG", quality=60, optimize=True, progressive=True)
    im_io.seek(0)
    im_io.name = "testimage.jpg"
    return im_io


def small_uploaded_file(small_jpeg_io):
    simple_png = SimpleUploadedFile(
        name="test.png", content=small_jpeg_io.read(), content_type="image/png"
    )
    small_jpeg_io.seek(0)
    return simple_png

In [5]:
simple_png = small_uploaded_file(small_jpeg_io())

In [6]:
from django.conf import settings

print(settings.DATABASES)

{'default': {'ENGINE': 'django.db.backends.sqlite3', 'NAME': '/Users/jochen/projects/wagtail_srcset/example/db.sqlite', 'ATOMIC_REQUESTS': False, 'AUTOCOMMIT': True, 'CONN_MAX_AGE': 0, 'OPTIONS': {}, 'TIME_ZONE': None, 'USER': '', 'PASSWORD': '', 'HOST': '', 'PORT': '', 'TEST': {'CHARSET': None, 'COLLATION': None, 'NAME': None, 'MIRROR': None}}}


In [7]:
image = WagtailImage(file=simple_png)
image.save()

# render template

In [8]:
template_text = """
{% load wagtail_srcset_tags %}

{% srcset_image img width-300 jpegquality-90 %}
"""

In [9]:
t = Template(template_text)
print(t.render(Context({"img": image})))

in srcset image:  <wagtail.images.templatetags.wagtailimages_tags.ImageNode object at 0x10f0d53a0>
srcset_string:  /media/images/test_ekYJKaE.width-2200.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-1100.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-768.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-500.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-300.jpegquality-90.jpg 200w
foobarbaz: 



<img alt="" height="200" src="/media/images/test_ekYJKaE.width-300.jpegquality-90.jpg" srcset="/media/images/test_ekYJKaE.width-2200.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-1100.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-768.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-500.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-300.jpegquality-90.jpg 200w" width="200">



In [10]:
template_text = """
{% load wagtail_srcset_tags %}

{% srcset_image img width-300 jpegquality-90 as thumbnail %}
thumbnail attrs: {{ thumbnail.attrs }}
image srcset_string: {{ img.srcset_string }}
thumbnail srcset: {{ thumbnail.srcset_string }}
"""

In [11]:
t = Template(template_text)
print(t.render(Context({"img": image})))

in srcset image:  <wagtail.images.templatetags.wagtailimages_tags.ImageNode object at 0x10f934a00>
srcset_string:  /media/images/test_ekYJKaE.width-2200.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-1100.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-768.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-500.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-300.jpegquality-90.jpg 200w
foobarbaz: 




thumbnail attrs:  alt="" height="200" src="/media/images/test_ekYJKaE.width-300.jpegquality-90.jpg" width="200"
image srcset_string: 
thumbnail srcset: /media/images/test_ekYJKaE.width-2200.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-1100.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-768.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-500.jpegquality-90.jpg 200w, /media/images/test_ekYJKaE.width-300.jpegquality-90.jpg 200w

